In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import linalg

### Controller Design (with ox_dot, oy_dot, oz_dot)

In [5]:
# Mass
m = 0.034

# Principle moments of inertia
J_x = 1.57e-05
J_y = 1.60e-05
J_z = 2.05e-05

# Acceleration of gravity
g = 9.81
l_pen = 320e-3 # m

p_eq = [m, J_x, J_y, J_z]

In [6]:
def Rotate(alpha, beta, gamma):
    Rz = sym.Matrix([[sym.cos(alpha), -sym.sin(alpha), 0],
                 [sym.sin(alpha), sym.cos(alpha), 0],
                 [0, 0, 1]])

    Ry = sym.Matrix([[sym.cos(beta), 0, sym.sin(beta)],
                 [0, 1, 0],
                 [-sym.sin(beta), 0, sym.cos(beta)]])

    Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(gamma), -sym.sin(gamma)],
                 [0, sym.sin(gamma), sym.cos(gamma)]])

    R_VtoO = Rz * Ry * Rx

    return R_VtoO

def N_func(alpha, beta, gamma):
    Ninv = sym.Matrix([[sym.cos(beta)*sym.cos(gamma), -sym.sin(gamma), 0],
                       [sym.cos(beta)*sym.sin(gamma), sym.cos(gamma), 0],
                       [-sym.sin(beta), 0, 1]])

    # N = sym.simplify(Ninv.inv())
    N = Ninv.inv()
    return N

In [7]:
o_x, o_y, o_z, alpha, beta, gamma, wx, wy, wz, ox_dot, oy_dot, oz_dot= sym.symbols(r'o_x o_y o_z \alpha \beta \gamma w_x w_y w_z ox_{dot} oy_{dot} oz_{dot}')
tau_x, tau_y, tau_z, f_z = sym.symbols('tau_x, tau_y, tau_z, f_z')
m, Jx, Jy, Jz = sym.symbols('m J_x J_y J_z')
J_in1 = sym.diag(Jx, Jy, Jz)
tau_in1 = sym.Matrix([tau_x, tau_y, tau_z])
w_01in1 = sym.Matrix([wx, wy, wz])
# state: [o_x, o_x_dot, o_y, o_y_dot, o_z, o_z_dot, alpha, beta, gamma, wx, wy, wz]
# inputs: [tau_x, tau_y, tau_z, f_z]
Rotation_matrix = Rotate(alpha, beta, gamma)
N = N_func(alpha, beta, gamma)
second_derivatives = Rotation_matrix * sym.Matrix([0, 0, f_z])/m + sym.Matrix([0, 0, -g])
angle_derivatives = N * sym.Matrix([wx, wy, wz]) # [gamma_dot, beta_dot, alpha_dot]
w_derivatives = J_in1.inv() * (tau_in1 - w_01in1.cross(J_in1 * w_01in1))
f_sym = sym.zeros(12,1)
f_sym[0] = ox_dot
f_sym[1] = second_derivatives[0]
f_sym[2] = oy_dot
f_sym[3] = second_derivatives[1]
f_sym[4] = oz_dot
f_sym[5] = second_derivatives[2]
f_sym[6] = angle_derivatives[2]
f_sym[7] = angle_derivatives[1]
f_sym[8] = angle_derivatives[0]
f_sym[9] = w_derivatives[0]
f_sym[10] = w_derivatives[1]
f_sym[11] = w_derivatives[2]
f_sym

Matrix([
[                                                                                                                                                                     ox_{dot}],
[                                                                                                         f_z*(sin(\alpha)*sin(\gamma) + sin(\beta)*cos(\alpha)*cos(\gamma))/m],
[                                                                                                                                                                     oy_{dot}],
[                                                                                                         f_z*(sin(\alpha)*sin(\beta)*cos(\gamma) - sin(\gamma)*cos(\alpha))/m],
[                                                                                                                                                                     oz_{dot}],
[                                                                                                         

In [8]:
s = [o_x, ox_dot, o_y, oy_dot, o_z, oz_dot, alpha, beta, gamma, wx, wy, wz]
i = [tau_x, tau_y, tau_z, f_z]
p = [m, Jx, Jy, Jz]

In [9]:
# Mass
m = 0.034

# Principle moments of inertia
J_x = 1.57e-05
J_y = 1.60e-05
J_z = 2.05e-05

# Acceleration of gravity
g = 9.81
l_pen = 320e-3 # m

p_eq = [m, J_x, J_y, J_z]

In [10]:
f = sym.lambdify(s + i + p, f_sym)
f

<function _lambdifygenerated(o_x, ox_dot, o_y, oy_dot, o_z, oz_dot, Dummy_112, Dummy_111, _Dummy_110, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z, m, J_x, J_y, J_z)>

In [11]:
s_eq = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
i_eq = [0., 0., 0., g*(m)]

In [12]:
print(f(*s_eq, *i_eq, *p_eq))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [13]:
A_sym = f_sym.jacobian(s)
B_sym = f_sym.jacobian(i)

A_num = sym.lambdify(s + i + p, A_sym)
B_num = sym.lambdify(s + i + p, B_sym)

A = A_num(*s_eq, *i_eq, *p_eq)
B = B_num(*s_eq, *i_eq, *p_eq)

In [14]:
# state: [o_x, o_x_dot, o_y, o_y_dot, o_z, o_z_dot, alpha, beta, gamma, wx, wy, wz, r, s, rdot, sdot]
############# JANK SHIT    ##############################
A_mod = np.vstack((np.hstack((A, np.zeros((12, 4)))), np.zeros((4,16))))
A_mod[12,14] = 1.0
A_mod[13,15] = 1.0
A_mod[14,12] = g/l_pen
A_mod[14, 7] = -g
A_mod[15,13] = g/l_pen
A_mod[15, 8] = g

B_mod = np.vstack((B, np.zeros((4,4))))

In [18]:
A_str = np.array2string(A_mod,
                        formatter={'float_kind': lambda x: f'{x:5.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'A = {A_str}')

A = [[ 0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  9.81  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 -9.81  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00 -0.00 -0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 -0.00 -0.00  1.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  

In [19]:
B_str = np.array2string(B_mod,
                        formatter={'float_kind': lambda x: f'{x:10.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)
B
print(f'B = {B_str}')

B = [[      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00      29.41]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [  63694.27       0.00       0.00       0.00]
     [      0.00   62500.00       0.00       0.00]
     [      0.00       0.00   48780.49       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]]


In [28]:
Q = np.diag([
    50., # o_x
    5., # o_x_dot
    50., # o_y
    5., # o_y_dot
    1., # o_z
    1., # o_z_dot
    1., # alpha
    5., # beta
    5., # gamma
    1., # wx
    1., # wy
    1., # wz
    10., # r
    10., # s
    1., # rdot
    1.  # sdot
])

R = np.diag([
    1E6, #tau_x
    1E6,#tau_y
    1E6,
    10,
])

def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K


In [29]:
K = lqr(A_mod, B_mod, Q, R)

K

array([[-1.12405489e-14, -6.85017159e-15,  7.07106781e-03,
         6.99189063e-03,  8.31378676e-18,  1.19192739e-17,
         6.28650112e-17,  6.63212081e-15,  2.38516552e-02,
         1.32247570e-03,  8.44293109e-17, -9.03919648e-19,
        -9.10368702e-14,  1.69670327e-01, -1.66436115e-14,
         3.06550575e-02],
       [-7.07106781e-03, -6.99387830e-03,  8.41036473e-15,
         6.30006278e-15,  1.16557878e-18,  2.06783201e-18,
         8.44216770e-17,  2.39136220e-02,  2.65800641e-15,
         8.28462614e-17,  1.32862181e-03, -1.23179949e-16,
        -1.69924234e-01,  7.72495206e-14, -3.07008991e-02,
         1.38445284e-14],
       [ 6.54647245e-15,  5.29267383e-15, -1.53647680e-15,
        -1.86456417e-15,  1.20419851e-17,  1.41986399e-17,
         1.00000000e-03, -7.43161715e-15, -2.03918512e-15,
        -6.92270169e-19, -9.61404483e-17,  1.02029408e-03,
         8.29816523e-14, -2.76512309e-14,  1.50732466e-14,
        -5.00335163e-15],
       [-4.67476938e-15, -6.65146898e